Avant de lancer ce NB, je lance le terminal Anaconda et je copie colle : 
mlflow server --host 127.0.0.1 --port 8080 

Ca lance MLFlow

Pour ce projet, nous avons suivi le schéma présenté dans cet article:
https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420

# 1 - Imports


In [13]:
import mlflow
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, fbeta_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler


from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import lightgbm as lgb

from mlflow.models import infer_signature
from lightgbm import early_stopping

import shap
# Memory management
import gc 

In [14]:
# démarrage du tracking
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [15]:
# Configuration de la connexion MLflow
mlflow.utils.rest_utils.DEFAULT_RETRIES = 10
mlflow.utils.rest_utils.DEFAULT_BACKOFF_FACTOR = 0.2
mlflow.utils.rest_utils.DEFAULT_TIMEOUT = 60

In [16]:
# Vérifiez que le serveur MLflow est accessible
try:
    mlflow.get_experiment_by_name("MLflow Credit_Scoring - Projet_7")
    print("Le serveur MLflow est accessible.")
except Exception as e:
    print(f"Erreur de connexion au serveur MLflow: {e}")

Le serveur MLflow est accessible.


In [17]:
train_reduced=pd.read_csv('train_reduced.csv')

In [18]:
train_reduced.head()

,EXT_SOURCE_1,CREDIT_TERM,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_EMPLOYED,DAYS_BIRTH,client_installments_AMT_PAYMENT_min_sum,AMT_ANNUITY,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_ENDDATE_max,...,client_credit_CNT_DRAWINGS_POS_CURRENT_max_sum,client_credit_AMT_PAYMENT_CURRENT_min_sum,previous_NAME_SELLER_INDUSTRY_Industry_mean,client_credit_AMT_INST_MIN_REGULARITY_max_sum,previous_NAME_GOODS_CATEGORY_Sport and Leisure_mean,client_credit_CNT_DRAWINGS_POS_CURRENT_min_sum,client_credit_NAME_CONTRACT_STATUS_Completed_mean_min,OCCUPATION_TYPE_Drivers,TARGET,SK_ID_CURR
0,0.083037,0.060749,0.262949,0.139376,-637.0,9461,175783.73,24700.5,-103.0,780.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,False,True,100002
1,0.311267,0.027598,0.622246,NaN,-1188.0,16765,1154108.20,35698.5,-606.0,1216.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,False,False,100003
2,NaN,0.050000,0.555912,0.729567,-225.0,19046,16071.75,6750.0,-408.0,-382.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,False,False,100004
3,NaN,0.094941,0.650442,NaN,-3039.0,19005,994476.70,29686.5,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,100006
4,NaN,0.042623,0.322738,NaN,-3038.0,19932,483756.38,21865.5,-1149.0,-783.0,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,False,False,100007


In [19]:
test_reduced=pd.read_csv('test_reduced.csv')

In [20]:
train_reduced.shape, test_reduced.shape

((307511, 349), (48744, 348))

In [21]:
train_reduced=train_reduced.drop(['AMT_CREDIT', 'AMT_ANNUITY', 'CREDIT_TERM'], axis=1)
test_reduced=test_reduced.drop(['AMT_CREDIT', 'AMT_ANNUITY', 'CREDIT_TERM'], axis=1)

In [22]:
train_reduced.shape, test_reduced.shape

((307511, 346), (48744, 345))

# 2 - Préparation des données

In [ ]:
train_r, test_r = train_test_split(train_reduced, test_size=0.2, random_state=42)

# Extraire les identifiants, les cibles et les caractéristiques pour les ensembles d'entraînement et de test
X_train_r = train_r.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_train_r = train_r['TARGET']
id_train_r = train_r['SK_ID_CURR']

X_test_r = test_r.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y_test_r = test_r['TARGET']
id_test_r = test_r['SK_ID_CURR']
    

In [ ]:
X_train_r.shape, X_test_r.shape, y_train_r.shape, y_test_r.shape, id_train_r.shape, id_test_r.shape

In [ ]:
def prepare_modelisation (X_train, X_test) :

    # Create a label encoder object
    le = LabelEncoder()
    
    # Iterate through the columns and label encode if object type and nunique <=2
    for col in X_train.columns:
        if X_train[col].dtype == 'object' and len(list(X_train[col].unique())) <= 2:
            # Apply the label encoder to both training and test sets
            X_train[col] = le.fit_transform(X_train[col])
            X_test[col] = le.transform(X_test[col])

               
    # one-hot encoding of categorical variables
    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)

    # Assurons-nous que X_train et X_test ont les mêmes colonnes
    X_train, X_test = X_train.align(X_test, join='inner', axis=1)
    
   
    # Median imputation of missing values
    imputer = SimpleImputer(strategy = 'median')

    # Scale each feature to 0-1
    scaler = MinMaxScaler(feature_range = (0, 1))

    X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

   
    return X_train, X_test

In [ ]:
X_train_r, X_test_r=prepare_modelisation (X_train_r, X_test_r)

In [ ]:
X_train_r.shape, X_test_r.shape, y_train_r.shape, y_test_r.shape, id_train_r.shape, id_test_r.shape

# **FONCTION DU NB 1 A SUPPRIMER QUAND ON GROUPERA LES NB**

In [ ]:
def custom_f1(y_true, y_pred_proba):
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_f1 = 0
    best_cost = float('inf')
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        # Calculer les coûts
        fn = np.sum((y_true == 1) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        cost = fn * 10 + fp
        
        # Sélectionner le seuil basé sur le coût le plus bas
        if cost < best_cost:
            best_f1 = f1
            best_threshold = threshold
            best_cost = cost
    
    return best_f1

In [ ]:


def custom_recall(y_true, y_pred_proba):
    thresholds = np.linspace(0, 1, 100)
    best_threshold = 0.5
    best_recall = 0
    best_cost = float('inf')
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        recall = recall_score(y_true, y_pred)
        # Calculer les coûts
        fn = np.sum((y_true == 1) & (y_pred == 0))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        cost = fn * 10 + fp
        
        # Sélectionner le seuil basé sur le coût le plus bas
        if cost < best_cost:
            best_recall = recall
            best_threshold = threshold
            best_cost = cost
    
    return best_recall

In [ ]:
# Fonction pour run la grid search. J'y ajoute le fit_params dans le quel j'ajouterai l'early stopping pour le lgbm

def run_grid_search(X_train, y_train, model, param_grid, fit_params=None, train_size=1.0):
    # Définir les scorers personnalisés
    scorers = {
        'AUC': 'roc_auc',
        'F1_opt': make_scorer(custom_f1, needs_proba=True),
        'recall_opt': make_scorer(custom_recall, needs_proba=True)
    }

    # Créer un pipeline avec SMOTE et le modèle
    pipeline = Pipeline([
        ('sampling', SMOTE(random_state=42)),
        ('model', model)
    ])

    # Mettre à jour le param_grid pour correspondre au pipeline
    # Les paramètres du modèle doivent être préfixés par 'model__'
    param_grid = {f'model__{key}': value for key, value in param_grid.items()}

    grid_search = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        scoring=scorers,
        refit='AUC',
        cv=2,
        verbose=3,
        return_train_score=True
    )

     # Fractionnement des données si train_size < 1.0
    if train_size < 1.0:
        X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=train_size, random_state=42)
        data_to_fit = (X_train_sample, y_train_sample)
    else:
        data_to_fit = (X_train, y_train)

    # Entraînement
    start_time = time.time()
    grid_search.fit(*data_to_fit)
    end_time = time.time()

    execution_time = round(end_time - start_time, 2)
    print(f"Le temps d'execution est de {execution_time} secondes.")

    return {
        'best_model' : grid_search.best_estimator_,
        'best_params_': grid_search.best_params_,
        'best_score_': grid_search.best_score_,
        'cv_results_': grid_search.cv_results_
    }

In [ ]:
def extract_best_auc_result(results):
    """
    Extracts the best model configuration based on AUC score from the cv_results of a GridSearchCV.
    
    Parameters:
    - results (dict): A dictionary output from the run_grid_search function containing 'cv_results_'.
    
    Returns:
    - DataFrame: A DataFrame containing the best scoring row based on the AUC rank.
    """
    # Conversion of cv_results_ into a DataFrame
    cv_results = results['cv_results_']
    df_cv_results = pd.DataFrame(cv_results)
    
    # Sorting the DataFrame by the rank of the AUC test scores
    df_cv_results = df_cv_results.sort_values(by="rank_test_AUC", ascending=True)
    
    # Returning the top entry
    return df_cv_results.head(1)

In [ ]:
def plot_auc_scores(best_auc_result):
    """
    Plots the AUC scores from cross-validation folds for the best model configuration based on rank.
    
    Parameters:
    - best_auc_result (DataFrame): A DataFrame with the top row from cv_results_ sorted by AUC.
    - full_cv_results (DataFrame): The full cv_results DataFrame to extract mean scores for rank 1.
    """
    # Extract the AUC scores for each fold for the best AUC configuration
    df_train_results_auc = best_auc_result[[
        'split0_train_AUC', 
        'split1_train_AUC', 
        # 'split2_test_AUC', 
        # 'split3_test_AUC', 
        # 'split4_test_AUC'
    ]][best_auc_result['rank_test_AUC'] == 1].values
    
    df_test_results_auc = best_auc_result[[
        'split0_test_AUC', 
        'split1_test_AUC', 
        # 'split2_test_AUC', 
        # 'split3_test_AUC', 
        # 'split4_test_AUC'
    ]][best_auc_result['rank_test_AUC'] == 1].values
    
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Plot the train scores
    ax.plot(range(0, 2), df_test_results_auc.reshape(-1), label='Scores de validation')
    
    # Plot the test scores
    ax.plot(range(0, 2), df_train_results_auc.reshape(-1), label='Scores de train')

    # # Plot the average for the combinations with rank_test_AUC == 1
    # mean_scores_rank_1 = best_auc_result.loc[best_auc_result['rank_test_AUC'] == 1, 'mean_test_AUC']
    # ax.axhline(y=mean_scores_rank_1.values[0], color='g', linestyle='--', label='Moyenne (rank 1)')
    
    # Set the properties of the axis
    ax.set_xticks(range(0, 2))
    ax.set_xlabel("Folds de cross-validation")
    ax.set_ylabel('AUC')
    ax.set_title("AUC de chaque fold \n pour la combinaison d'hyperparamètres \n qui arrive au rang 1 en terme d'AUC", fontsize=15, weight="bold", fontname="Impact", color="#0e2452")
    
    # Add a legend
    ax.legend()

    fig.tight_layout()
    
    plt.show()


In [ ]:
# fonction qui va établir un seuil pour convertir les probabiliiéts en étiquettes de classe binaire. 
# à partir de la matrice de confusion, on va calculer le coût

def calculate_cost_threshold(y_true, probas, threshold, cost_fn, cost_fp):
    y_pred = (probas >= threshold).astype(int) # transforme les probas en prédictions binaires en utilisant un seuil. 
                                                # Les probabilités >= au seuil sont marquées comme 1 (positives),
                                                # les autres comme 0 (négatives).

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel() # .ravel() convertit la matrice 2x2 en un tableau à une dimension [tn, fp, fn, tp]
                                                            # permet une extraction facile de chaque valeur
    return cost_fn * fn + cost_fp * fp

In [ ]:
def evaluate_model (X_train, y_train, X_test, y_test, best_model):

    start_predict_time = time.time()
    # Predict on the training data
    y_train_pred_proba = best_model.predict_proba(X_train)[:, 1]
    
    # Predict on the test data
    y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

    end_predict_time = time.time()

    # Calculate AUC for training and test data
    auc_train = round(roc_auc_score(y_train, y_train_pred_proba),2)
    auc_test = round(roc_auc_score(y_test, y_test_pred_proba),2)

    # Find the optimal threshold for cost function on the training data
    thresholds = np.linspace(0, 1, 100)
    costs = [calculate_cost_threshold(y_test, y_test_pred_proba, thr, cost_fn=10, cost_fp=1) for thr in thresholds]
    optimal_threshold = round(thresholds[np.argmin(costs)],2) 

    # Metrics at the optimal threshold
    y_test_pred_opt = (y_test_pred_proba >= optimal_threshold).astype(int)
    
    f1_score_1_test = f1_score(y_test, y_test_pred_opt)
    recall_1_test = recall_score(y_test, y_test_pred_opt)
    accuracy_test = accuracy_score(y_test, y_test_pred_opt)
    
    return {
        'predicting_execution_time': round(end_predict_time - start_predict_time, 2),
        'auc_train': auc_train,
        'auc_test': auc_test,
        'optimal_threshold': optimal_threshold,
        'f1_score_1_test': round(f1_score_1_test,2),
        'recall_1_test': round(recall_1_test,2),
        'accuracy_test': round(accuracy_test,2)
    }

In [ ]:
def show_feature_importances(importances, features):
    # Création du DataFrame

    importances = importances.flatten()
    
    feature_importances = pd.DataFrame({
        'feature': features, 
        'importance': importances
    })
    
    # Tri et sélection des 20 caractéristiques les plus importantes
    feature_importances_sorted = feature_importances.sort_values(by='importance', ascending=False).reset_index(drop=True)
    feature_importances_sorted = feature_importances_sorted.head(20)
    
    # Normalisation des importances
    feature_importances_sorted['importance_normalized'] = feature_importances_sorted['importance'] / feature_importances_sorted['importance'].sum()
    
    # Création du graphique à barres horizontales
    plt.figure(figsize=(10, 6))
    ax = plt.subplot()
    
    # Inversion de l'index pour afficher la plus importante en haut
    indices = list(reversed(list(feature_importances_sorted.index)))
    ax.barh(indices, feature_importances_sorted['importance_normalized'], align='center', edgecolor='k')
    
    # Définition des étiquettes y
    ax.set_yticks(indices)
    ax.set_yticklabels(feature_importances_sorted['feature'])
    
    # Étiquetage du graphique
    plt.xlabel('Normalized Importance')
    plt.title('Top 20 Feature Importances')
    plt.show()
    
    return feature_importances_sorted

# Exemple d'utilisation
# importances = [valeur1, valeur2, ..., valeurN] # remplacez par vos valeurs d'importance réelles
# features = ['nom1', 'nom2', ..., 'nomN'] # remplacez par vos noms de caractéristiques réels
# feature_importances_sorted = plot_feature_importances(importances, features)


# **FIN DES FONCTIONS DU NB 1 A SUPPRIMER QUAND ON GROUPERA LES NB**

# 3 - Nouvelle modélisation LGBM avec features = 95% de l'importance

In [ ]:
model_LGBM_2 = lgb.LGBMClassifier(objective='binary',
                                boosting_type = 'goss',
                                # class_weight = 'balanced',
                                random_state = 50)
        
param_grid_LGBM_2 = {
    'num_leaves': [35,40],
    'n_estimators' : [150,200],   
    'learning_rate' : [0.07, 0.1],
}

# Paramètres pour early_stopping
fit_params = {
    'eval_metric': 'auc',
    'callbacks': [early_stopping(stopping_rounds=50)]
}

In [ ]:
# Execution de la fonction
start_training_time = time.time()

results_LGBM_2 = run_grid_search(X_train_r, y_train_r, model_LGBM_2, param_grid_LGBM_2,fit_params)

end_training_time = time.time()

In [ ]:
training_execution_time = round(end_training_time - start_training_time,2)  
print(f"Le temps d'exécution est de {training_execution_time} secondes.")

In [ ]:
best_model_lgbm_2 = results_LGBM_2['best_model']
best_params_lgbm_2 = results_LGBM_2['best_params_']
best_score_lgbm_2 = results_LGBM_2['best_score_']
cv_results_lgbm_2 = results_LGBM_2['cv_results_']

print("Best Params:", best_params_lgbm_2)
print("Best AUC:", best_score_lgbm_2)

In [ ]:
best_auc_result_LGBM_2 = extract_best_auc_result(results_LGBM_2)
best_auc_result_LGBM_2.head(1)

In [ ]:
plot_auc_scores(best_auc_result_LGBM_2)

In [ ]:
result_lgbm_2=evaluate_model (X_train_r, y_train_r, X_test_r, y_test_r, best_model_lgbm_2)
result_lgbm_2

In [ ]:
best_model_lgbm_2.named_steps['model']

In [ ]:
# Utiliser un échantillon de 10 lignes comme exemple d'entrée
input_example = X_train_r.sample(n=10, random_state=42)

# 4 - Enregistrement du modèle

In [ ]:
# Création d'une nouvelle expérience MLflow
mlflow.set_experiment("MLflow Credit_Scoring - Projet_7")

# Commencer une session MLflow
with mlflow.start_run():
    # Log les hyperparameters que j'ai déclarés au dessus
    mlflow.log_params(best_params_lgbm_2)

    # Log les métriques
    
    mlflow.log_metric("optimal_threshold", result_lgbm_2['optimal_threshold'])

    mlflow.log_metric("recall_1_test", result_lgbm_2['recall_1_test'])
    mlflow.log_metric("f1_score_1_test", result_lgbm_2['f1_score_1_test'])
    mlflow.log_metric("accuracy du test", result_lgbm_2['accuracy_test'])
    
    mlflow.log_metric("auc_train", result_lgbm_2['auc_train'])
    mlflow.log_metric("auc_test", result_lgbm_2['auc_test'])
    
    mlflow.log_metric("temps_fit", training_execution_time)
    mlflow.log_metric("temps_predict", result_lgbm_2['predicting_execution_time'])


    # Définir un tag pour se rappeler l'objet de cette session
    mlflow.set_tag("Training Info", "LGBM with 95% importance features")

    # Infer the model signature
    signature = infer_signature(X_train_r, best_model_lgbm_2.predict(X_train_r)) # infer_signature génère automatiquement une "signature" 
                                                              # qui décrit les entrées et les sorties du modèle. 
                                                              # Cela inclut les types de données et les formats attendus par le modèle, 
                                                              # facilitant ainsi l'intégration et la réutilisation du modèle 
                                                              # dans différents environnements.

    
    # Log le model
    model_info = mlflow.lightgbm.log_model(
        lgb_model=best_model_lgbm_2.named_steps['model'], # le modèle
        artifact_path="lgbm_classifier_model", # le chemin où le modèle sera enregistré
        signature=signature, # la signature du modèle
        input_example=input_example, # un exemple d'input pour montrer comment invoquer le modèle
        registered_model_name="scoring-credit-lgbm_classifier", # nom sous lequel le modèle est enregistré dans le registre de modèles MLflow
    )




# 5 - Interprétation des résultats

## 5.1 - Feature Importance

In [ ]:
importances = best_model_lgbm_2.named_steps['model'].feature_importances_
features= X_train_r.columns

In [ ]:
importances

In [ ]:
X_train_r.head(1)

In [ ]:
show_feature_importances(importances, features)

In [ ]:
feature_importances_sorted=show_feature_importances(importances, features)

In [ ]:
feature_importances_sorted

In [ ]:
feature_importances_sorted.to_csv('feature_importances_sorted_lgbm_2.csv', index=False)

## 5.2 - Shap Values

### 5.2.1 - Interprétation globale du test

In [ ]:
# on s'assure que le meilleur modèle est extrait de la pipeline
best_model=best_model_lgbm_2.named_steps['model']

In [ ]:
# # Comme le SMOTE est appliqué dans le pipeline, nous devons transformer les données X_test_r avant d'appliquer SHAP
# # Pour cela, nous utilisons le transformateur dans le pipeline avant le modèle
# data_for_shap = results_LGBM_2['best_model'].named_steps['sampling'].fit_resample(X_test_r, y_test_r)

In [ ]:
# # Extraction des données transformées qui sont maintenant suréchantillonnées
# X_test_resampled, y_test_resampled = data_for_shap

In [ ]:
# Initialisation de l'Explainer SHAP avec le meilleur modèle obtenu sur le jeu de données de test original
explainer = shap.Explainer(best_model)

In [ ]:
# Calcul des valeurs SHAP sur les données de test
shap_values = explainer.shap_values(X_test_r)

In [ ]:
# Créer un graphique récapitulatif SHAP pour l'ensemble des données transformées
shap.summary_plot(shap_values, X_test_r, 
                  plot_type="bar",
                 max_display=10)

Si on regarde ce qui se passe pour la classe 1 :

In [ ]:
shap_values_class1 = shap_values[1]

In [ ]:
# Graphique récapitulatif SHAP pour la classe 1
shap.summary_plot(shap_values_class1, 
                  X_test_r, 
                  max_display=10)

### 5.2.2 - Interprétation locale

In [ ]:
# verif
train_reduced[train_reduced['SK_ID_CURR']==100002]

In [ ]:
test_r[test_r['SK_ID_CURR']==100002]

In [ ]:
# reconstruction du test
# test_dataset = pd.concat([X_test_r, y_test_r.reset_index(drop=True), id_test_r.reset_index(drop=True)], axis=1)
# # verif
# test_dataset[test_dataset['SK_ID_CURR']==100002]

In [ ]:
# predict proba avec le best model
y_test_pred_proba = best_model_lgbm_2.predict_proba(X_test_r)[:, 1]

In [ ]:
# Find the optimal threshold for cost function on the training data
thresholds = np.linspace(0, 1, 100)
costs = [calculate_cost_threshold(y_test_r, y_test_pred_proba, thr, cost_fn=10, cost_fp=1) for thr in thresholds]
optimal_threshold = round(thresholds[np.argmin(costs)],2) 

# Metrics at the optimal threshold
y_test_pred_opt = (y_test_pred_proba >= optimal_threshold).astype(int)

In [ ]:
test_r['target_pred']=y_test_pred_opt
test_r['proba']=y_test_pred_proba

test_r[test_r['SK_ID_CURR']==100002]

In [ ]:
pd.crosstab(y_test_r, y_test_pred_opt, rownames=['Classes réelles'], colnames=['Classes prédites'])

In [ ]:
print(classification_report(y_test_r, y_test_pred_opt))

#### 5.2.2.1 - Interprétation d'un individu prédit en classe 1

In [ ]:
test_r[test_r['target_pred']==1].head()

In [ ]:
# ID de l'individu à analyser
specific_id = 384575

In [ ]:
# Trouver l'index de cet individu dans le jeu de données de test
index = test_r[test_r['SK_ID_CURR'] == specific_id].index[0]

In [ ]:
# Réinitialiser les index de X_test_r pour obtenir des index consécutifs
X_test_r_reset = X_test_r.reset_index(drop=True)

# Réinitialiser les index de test_r pour obtenir des index consécutifs
test_r_reset = test_r.reset_index(drop=True)

Cette étape assure que les index sont consécutifs et commencent à 0, ce qui facilite l'alignement.

In [ ]:
# Trouver le nouvel index relatif dans le jeu de données réinitialisé
relative_index = test_r_reset[test_r_reset['SK_ID_CURR'] == specific_id].index[0]

Cette ligne de code trouve l'index relatif de l'individu spécifique dans test_r_reset.

In [ ]:
# Vérifier les dimensions de shap_values et X_test_r_reset 
print("Dimensions des shap_values pour la classe 1:", shap_values[1].shape)
print("Dimensions de X_test_r_reset:", X_test_r_reset.shape)

In [ ]:
# Extraire les valeurs SHAP pour cet individu spécifique pour la classe 1
individual_shap_values = shap_values[1][relative_index]

In [ ]:
# Créer le graphique waterfall pour cet individu
shap.plots.waterfall(shap.Explanation(values=individual_shap_values,
                                      base_values=explainer.expected_value[1],  # Assurez-vous de choisir la bonne valeur de base
                                      data=X_test_r_reset.iloc[relative_index],  # Données de l'individu
                                      feature_names=X_test_r_reset.columns.tolist()))  # Noms des caractéristiques

#### 5.2.2.2 - Interprétation d'un individu prédit en classe 0

In [ ]:
test_r[test_r['target_pred']==0].head()

In [ ]:
# ID de l'individu à analyser
specific_id = 214010

In [ ]:
# Trouver l'index de cet individu dans le jeu de données de test
index = test_r[test_r['SK_ID_CURR'] == specific_id].index[0]

In [ ]:
# Réinitialiser les index de X_test_r pour obtenir des index consécutifs
X_test_r_reset = X_test_r.reset_index(drop=True)

# Réinitialiser les index de test_r pour obtenir des index consécutifs
test_r_reset = test_r.reset_index(drop=True)

In [ ]:
# Trouver le nouvel index relatif dans le jeu de données réinitialisé
relative_index = test_r_reset[test_r_reset['SK_ID_CURR'] == specific_id].index[0]

In [ ]:
# Extraire les valeurs SHAP pour cet individu spécifique pour la classe 0
individual_shap_values = shap_values[0][relative_index]

In [ ]:
# Créer le graphique waterfall pour cet individu
shap.plots.waterfall(shap.Explanation(values=individual_shap_values,
                                      base_values=explainer.expected_value[1],  # Assurez-vous de choisir la bonne valeur de base
                                      data=X_test_r_reset.iloc[relative_index],  # Données de l'individu
                                      feature_names=X_test_r_reset.columns.tolist()))  # Noms des caractéristiques

In [ ]:
def generate_shap_waterfall(specific_id, shap_values, test_r, explainer, X_test_r):
    """
    Génère un graphique waterfall SHAP pour un individu spécifié par son ID.
    
    Parameters:
    - specific_id : int
        L'ID de l'individu pour lequel générer le graphique.
    - shap_values : list of numpy arrays
        Les valeurs SHAP pour les classes, typiquement [shap_values_class0, shap_values_class1].
    - test_r : DataFrame
        Le DataFrame contenant les identifiants des individus et les prédictions.
    - explainer : shap.Explainer
        L'explainer SHAP utilisé pour calculer les valeurs SHAP.
    - X_test_r : DataFrame
        Le DataFrame des données de test sur lequel les valeurs SHAP ont été calculées.
    """
    # Trouver l'index de l'individu dans le jeu de données de test
    index = test_r[test_r['SK_ID_CURR'] == specific_id].index[0]

    # Réinitialiser les index de X_test_r pour obtenir des index consécutifs
    X_test_r_reset = X_test_r.reset_index(drop=True)

    # Réinitialiser les index de test_r pour obtenir des index consécutifs
    test_r_reset = test_r.reset_index(drop=True)

    # Trouver le nouvel index relatif dans le jeu de données réinitialisé
    relative_index = test_r_reset[test_r_reset['SK_ID_CURR'] == specific_id].index[0]

    # Extraire les valeurs SHAP pour cet individu spécifique
    predicted_class = test_r.loc[test_r.index[relative_index], 'target_pred']
    individual_shap_values = shap_values[predicted_class][relative_index]

    # Créer le graphique waterfall pour cet individu
    shap.plots.waterfall(shap.Explanation(values=individual_shap_values,
                                          base_values=explainer.expected_value[predicted_class],  # Assurez-vous de choisir la bonne valeur de base
                                          data=X_test_r_reset.iloc[relative_index],  # Données de l'individu
                                          feature_names=X_test_r_reset.columns.tolist()))  # Noms des caractéristiques


In [ ]:
# Pour un individu prédit en classe 1
generate_shap_waterfall(384575, shap_values, test_r, explainer, X_test_r)

# Pour un individu prédit en classe 0
generate_shap_waterfall(214010, shap_values, test_r, explainer, X_test_r)

CONCLUSION : la suppression des variables AMT_CREDIT, AMT_ANNUITY, CREDIT_TERM n'a pas affecté la performance du modèle

Pour la suite, je me suis basée sur : https://app.livestorm.co/openclassrooms-1/deployez-une-api-de-prediction/live?s=021e19b0-cd31-4872-8b6f-b71209d05664#/chat

# 6 - Sérialisation du modèle

Sérialisation = le fait de convertir un objet qui est stocké de façon numérique sur notre mémoire ram. et le stocker sous forme de binaire sur notre ordiinateur. la pipeline était dans notre mémoire vive. On veut la stocker sous forme de fichier pour pouvoir la réutiliser plus tard

On peut sérialser en pickle ou en joblib. Avec Scikit Learn, on recommande joblib

In [ ]:
import joblib

In [ ]:
joblib.dump(best_model_lgbm_2, 'credit_scoring.joblib')

# 7 - Déploiement du modèle en local avec Streamlit

Il faut créer un nouveau fichier Python (par exemple app.py) pour le déploiement avec Streamlit. Je l'ai créé et enregistrer sous ce nom : 'app.py'

J'ai systématiquement des erreurs. Ci-dessous toutes les vérif que j'ai faites et c'est ok.